In [50]:
# import

from tqdm.notebook import tqdm  # 진행과정 시각화
from datetime import timedelta  # 시간날짜
import time
import re
import os
import datetime
import pandas as pd
import numpy as np
import gc

import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup  # 크롤링

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important; }</style>'))

In [51]:
def del_outword(string):
    '''크롤링 후 이모티콘, \u200b과 같은 문자가 아닌 것 제거
    [출처](https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python)
    '''
    #이모지제거
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

    #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
    han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~"^_\n\r#\ufeff\u200d\u200b\u7643\ufffd\u682e\u62c4]+')
    
    string = emoji_pattern.sub(r'',string)
    string = han.sub(r'',string)
    
    return string

def time_change(x):
    '''검색결과에 있는 문자를 날짜로 변경
    네이버 블로그 크롤링시 작성일자가 1분전/1시간전/1일전으로 나타는 경우가 가끔씩 존재
    이러한 패턴을 일반적인 YY-MM-DD형태로 변환함
    그리고 월별 계산이 중요하므로 모든 데이터는 1일로 통일
    
    Return
    -------
    x : datetime
        날짜형형태로 전환
        
    Example
    -------
    >>> x = "47분 전"
    >>> time_change(x)
    datetime.datetime(2020, 6, 1, 0, 0)
    
    >>> x2 = "2019.01.31"
    >>> time_change(x2)
    datetime.datetime(2019, 1, 1, 0, 0)
    '''
    
    min_pattern = re.compile('[0-9]+'+"분 전")
    hour_pattern = re.compile('[0-9]+'+"시간 전")
    day_pattern = re.compile('[0-9]+'+"일 전")

    today = datetime.datetime.today().date()
    # 일자
    if "일" in x :
        d = re.findall(day_pattern, x)[0][0]
        x = today - timedelta(days=int(d))
    elif "시간" in x:
        d = re.findall(hour_pattern, x)[0]
        x = today
    elif "분" in x:
        d = re.findall(min_pattern, x)[0]
        x = today
    elif x == "어제":
        x = today - timedelta(days=1)
    elif type(x) == str: # 'YY.NN.DD'형태
        x = datetime.datetime.strptime(x.replace('.', '-')[:-1], '%Y-%m-%d')
    
    #모든일자를 1일로 통일
    x = datetime.datetime.strptime(x.strftime('%Y-%m')+"-01","%Y-%m-%d")
    return x

def make_date_list(start_year,end_year,month_gap = 1,year_gap = 1, types = 'month'):
    '''검색에 사용할 날짜 list를 생성
    시작년도, 마지막년도를 입력하고 월별간격, 연별 간격을 입력하면
    검색에 사용할 "YYMMDD"형태의 날짜 list를 반환
    
    Parameters
    ----------
    start_year(int) : 검색을 시작할 연도
    end_year(int) : 검색을 종료할 연도
    month_gap(int) : 몇개월 단위로 검색할 것인지
    year_gap(int) : 몇년 단위로 검색할 것인지
    types(string) : 'year' or 'month'로 연간검색 혹은 월간검색지정
    
    Returns 
    -------
    start_date, end_date : list
        YYMMDD형태의 string형태의 날짜들은 반환한다
    
    Examples
    --------
    >>> make_date_list(start_year = 2010, end_year = 2011,month_gap = 1,year_gap = 1, types = 'month')
    start_date = ['20100101', '20100701', '20110101', '20110701']
    end_date = ['20100131', '20100731', '20110131', '20110731']
    
    >>> make_date_list(start_year = 2010, end_year = 2013,year_gap = 1, types = 'year')
    start_date = ['20100101', '20110101', '20120101', '20130101']
    end_date = ['20101231', '20111231', '20121231', '20131231']
    '''
    #일자
    month_gap = month_gap*100
    start_month = []
    end_month = []
    
    #MMDD 형태 추출
    for date in range(101,1202,month_gap):
        start_month.append(date)
    start_month = list(map(lambda x : '{:0>4}'.format(str(x)),start_month))

    for date in range(131,1232,month_gap):
        end_month.append(date)
    end_month = list(map(lambda x : '{:0>4}'.format(str(x)),end_month))

    start_date = []
    end_date = []
    
    #YYMMDD 형태로 추출
    if types.lower() == 'month':
        for year in range(start_year,end_year+1,year_gap):
            for date in start_month:
                start_date.append(str(year)+date)
            for date in end_month:
                end_date.append(str(year)+date)
    elif types.lower() == 'year':
        for year in range(start_year,end_year+1,year_gap):
            start_date.append(str(year)+'0101')
            end_date.append(str(year)+'1231')
            
        
    return start_date,end_date

In [52]:
keyword = '강릉'
date_option=8
date_from='20200101'
date_to='20201231'
base_url = f'https://search.naver.com/search.naver?date_from={date_from}&date_option={date_option}&date_to={date_to}&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={keyword}&sm=tab_pge&srchby=all&st=sim&where=post'
path = 'new_output'
#저장위치
os.makedirs('./new_output',exist_ok=True)

### 함수화
* 추가스크롤 발생시 추가로 url복사됨 (현재해결방안 중복된것 drop) -> 추후 특정위치 이상 발전이 없으면 중지하는 방안 필요

In [57]:
def new_crawling(keyword, path, date_from='', date_to='', save=True):
    '''네이버 블로그 검색결과가 드래그 방식으로 바뀜(기존 페이지방식)
    따라서 셀레니움으로 최대(1000건) 드래그후 작성일자, 제목, url을 가져온 후 
    request로 각 블로그에 접속하여 본문추출
    이 과정에서 네이버블로그가 아닌 블로그 제외됨(tistory, daum등)
    
    parameter
    ---------
    keyword(string) : 검색하고 싶은 키워드 "keyword +필수어" 형태로 필수단어 추가 가능
    path(string) : 저장할 폴더 설정
    date_from(YYYYMMDD) : (default = "") date_option이 8일때 사용 검색 시작일자를 지정
    date_to(YYYYMMDD) : (default = ""), date_option이 8일때 사용 검색 마지막일자를 지정
    save(bool) : (default = True)csv로 저장 여부 결정
    
    Returns
    ---------
    df : DataFrame
      post_dates title  full_text         url
    0 2010-01-01 title  [full_text]       http://blog.naver.com/PostView.nhn?blogId=wend...  
    
    '''
    
    base_url = f'https://search.naver.com/search.naver?date_from={date_from}&date_option=8&date_to={date_to}&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={keyword}&sm=tab_pge&srchby=all&st=sim&where=post'
    
    #임시저장
    blog_urls = dict()
    blog_urls['post_dates'] = []
    blog_urls['title'] = []
    blog_urls['url'] = []
    options = Options()
    
    # 크롬드라이버 작동 
    options.add_argument('--start-maximized') #전체화면
    options.add_argument('--headless') #headless(화면안켜고)
    driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)
    driver.get(base_url)

    #최대 아래로 스크롤
    count = 1
    b_blog_num = ''
    try :
        for i in range(0,40):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.5)
            count += 1
#             print(count)
            #스크롤 마지막 blog_num 비교
            source = driver.page_source
            soup = BeautifulSoup(source, 'html.parser')
            l_blog_num = soup.select('li[id^="sp_blog"]')[-1]['id']

            if b_blog_num == l_blog_num: #마지막 번호와 동일하면 스크롤 중지
                break
            else :
                b_blog_num = l_blog_num

        source = driver.page_source
        # print(count)
        driver.close()

        #각 정보 불러오기
        soup = BeautifulSoup(source, 'html.parser')

        #dates
        dates_list = soup.select('div > div.total_sub > span > span > span.etc_dsc_area > span')
        dates = pd.Series(dates_list).apply(lambda x : x.text)
        blog_urls['post_dates'] = blog_urls['post_dates'] + list(dates)

        #titles
        title_list = soup.select('div > a.api_txt_lines.total_tit')
        titles = pd.Series(title_list).apply(lambda x : x.text)
        blog_urls['title'] = blog_urls['title'] + list(titles)

        #link
        link_list = soup.select('div.total_dsc_wrap a.total_dsc')
        links = pd.Series(link_list).apply(lambda x : x.get('href'))
        blog_urls['url'] = blog_urls['url'] + list(links)

        #중복제거
        df = pd.DataFrame(blog_urls)
        df = df.drop_duplicates('url')
        blog_urls['url'] = df['url']
        ori_length = len(blog_urls['url'])

        # 본문가져오기
        output_error = []
        texts = []
        for i, url in tqdm(enumerate(blog_urls['url'])):
            count = i
            try:
                # 블로그 url안에 들어가기(아직 크롤링불가)
                get_blog_post_content_code = requests.get(url)
                get_blog_post_content_text = get_blog_post_content_code.text
                get_blog_post_content_soup = BeautifulSoup(
                    get_blog_post_content_text, 'lxml')

                # 크롤링가능한 url에 접속
                real_blog_post_url = "http://blog.naver.com" + get_blog_post_content_soup.select('#mainFrame')[0].get('src')
                get_real_blog_post_content_code = requests.get(real_blog_post_url)
                get_real_blog_post_content_text = get_real_blog_post_content_code.text
                get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')

                # 본문  (postviewarea 패턴과 se-main-container 2가지 유형이 있어 분리함)
                try:
                    blog_post_content = get_real_blog_post_content_soup.select('div.se-main-container')
                    if len(blog_post_content) == 0:
                        blog_post_content = get_real_blog_post_content_soup.select('div#postViewArea')
                        if len(blog_post_content) == 0:
                            blog_post_content = get_real_blog_post_content_soup.select('div.se_component_wrap.sect_dsc.__se_component_area')

                    string = ""
                    for sentence in blog_post_content[0].stripped_strings:
                        try :
                            string += " "+sentence.replace('\xa0', " ")
                        except :
                            pass
                        # 비언어 텍스트제거
                    string = del_outword(string)
                    # 공백에러대처
                    blank_check = string.replace(" ", "")
                    if len(blank_check) == 0:
                        texts.append(["X"])
                    else:
                        texts.append([string])
                    count += 1
                except Exception as ex:
                    print(ex)
                    texts.append(["X"])
                    output_error.append(count)
                    count += 1

            except Exception as ex:
    #             print('가져오기에러 {num}번째'.format(num=count), ex)
                output_error.append(count)
                count += 1
                pass

        #후처리
        df = df.drop(output_error) #에러제거
        df['full_text'] = texts
        df['post_dates'] = df['post_dates'].apply(lambda x : time_change(x)) #날짜 일반화
        last_length = len(output_error)
        print(f'검색된길이:{ori_length},제외된 길이:{last_length}')
        fin_length = df.shape[0]
        #저장
        if save == True:
            today = datetime.datetime.today().date()
            str(today)
            os.makedirs(f'./{path}/크롤링/{keyword}',exist_ok=True)
            df.to_csv(f'./{path}/크롤링/{keyword}/{date_from}~{date_to}_{keyword}_{fin_length}_{today}.csv',index=False)
    except Exception as ex:
        print(ex)
        pass
    return df

In [ ]:
start_dates, end_dates = make_date_list(2011,2020)

for date_num in tqdm(range(0, len(start_dates))):
    date_from = start_dates[date_num]
    date_to = end_dates[date_num]
    new_crawling(keyword, path, date_from, date_to,True)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: use options instead of chrome_options



검색된길이:540,제외된 길이:108



검색된길이:999,제외된 길이:144



검색된길이:480,제외된 길이:69



검색된길이:1000,제외된 길이:147


...


검색된길이:1000,제외된 길이:6



검색된길이:1000,제외된 길이:2



검색된길이:1000,제외된 길이:1



검색된길이:1000,제외된 길이:0



검색된길이:330,제외된 길이:0


### 이하 작업본

In [48]:
temp = pd.read_csv('./new_output/크롤링/강릉/20100101~20100131_강릉_173_2021-01-12.csv')

In [49]:
temp

,dates,title,urls,text
0,2010-01-01,향좋은 커피 공장 <테라로사> - 강릉 맛집,https://blog.naver.com/lcyyong5?Redirect=Log&l...,[' 강릉으로 여행을 가면 짬뽕을 먹던 바닷가에서 회를 먹든 향 좋은 커피를 만드는...
1,2010-01-01,<강원 강릉 카페> 언덕위의 바다 히피커피,https://blog.naver.com/dalpange98?Redirect=Log...,"[' 12월 31일, 해맞이 여행의 테마를 나름 커피여행으로 잡았다. 진즉부터 가보..."
2,2010-01-01,[강릉 맛집/교동 맛집]교동반점의 짬뽕,https://blog.naver.com/baeh76?Redirect=Log&log...,"["" 짬뽕을 좋아하는 1인으로서 강릉에 왔는데 어찌 그냥 갈수 있겠습니까... 대구..."
3,2010-01-01,강릉 홍길동전 박물관,https://blog.naver.com/neduk98?Redirect=Log&lo...,"["" 평소 박물관에서 시간 보내길 좋아하는 내가 강릉에서 연수받으면서 찾아간 '강릉..."
4,2010-01-01,[강릉맛집-만춘루] 강릉 짬뽕의 양대산맥중 하나랍니다.,https://blog.naver.com/dupolic?Redirect=Log&lo...,"["" 유치원 방학동안 강릉에 맡긴 아이를 다시 데리고 오기 위해 2009년 마지막날..."
...,...,...,...,...
822,2010-01-01,제왕산 겨울풍경 주목,https://blog.naver.com/lis0013?Redirect=Log&lo...,[' 백두대간 마루금 대관령 에서 동해를 바라보며 강원 강릉으로 내달린 한줄기 산맥...
823,2010-01-01,진화하는 부동산 투기 수법,https://blog.naver.com/juhodu?Redirect=Log&log...,[' real estate/ 부동산 관련 소득탈루 백태 Wealth Manageme...
824,2010-01-01,손목절단환자...,https://blog.naver.com/morningsix?Redirect=Log...,[' 주간업무를 마치고 구급대로 와서 밥을 배불리먹고 편히 쉬고있던 어느날 저녁.....
825,2010-01-01,솔 향 강 릉,https://blog.naver.com/kimhr9140?Redirect=Log&...,"["" 설화가 있어 더 정겨운 유적지 석천과 굴산사지 “처녀가 애를 낳아도 할 말이 ..."


In [34]:
#임시저장
blog_urls = dict()
blog_urls['dates'] = []
blog_urls['title'] = []
blog_urls['urls'] = []
options = Options()

# 크롬드라이버 작동 
options.add_argument('--start-maximized') #전체화면
# options.add_argument('--headless') #headless(화면안켜고)
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)
driver.get(base_url)

#최대 아래로 스크롤
count = 1
b_blog_num = ''
for i in range(0,40):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.5)
    count += 1
    print(count)
    #스크롤 마지막 blog_num 비교
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    l_blog_num = soup.select('li[id^="sp_blog"]')[-1]['id']
    
    if b_blog_num == l_blog_num: #마지막 번호와 동일하면 스크롤 중지
        break
    else :
        b_blog_num = l_blog_num

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


2
3


### 각 블로그 들어가서 url주소 따오는 방법(느림)

In [233]:
options = Options()
# 크롬드라이버 작동 
# os.chdir(',') #크롬드라이버위치
options.add_argument('--start-maximized') #전체화면
# options.add_argument('--headless') #headless(화면안켜고)
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#저장위치
blog_urls = dict()
blog_urls['dates'] = []
blog_urls['title'] = []
blog_urls['urls'] = []

#검색위치 
driver.get(base_url)
#최대 1천개
try :
    for i in tqdm_notebook(range(1,1001)):
        blog_url = driver.find_element_by_xpath(f'//*[@id="sp_blog_{i}"]/div/a[2]')
        #title
        title = blog_url.text
        blog_urls['title'].append(title)
        #date
        date = driver.find_element_by_xpath(f'//*[@id="sp_blog_{i}"]/div/div[1]/span/span/span[1]/span').text
        blog_urls['dates'].append(date)
        blog_url.click()
        driver.switch_to.window(driver.window_handles[-1]) #탭전환
        blog_urls['urls'].append(driver.current_url)
        driver.close()
        driver.switch_to.window(driver.window_handles[0]) #탭복구
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
except:
    pass

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


### 셀레니움으로 최대치(1천건)까지 아래로 스크롤 후 긁어오기
* 최대 1천이 아닐때 에러날것 -> 대안1 최대 1천건 스크롤(42번) 문제사항(문서가 짧으면 무의미한 스크롤) 대안필요

In [242]:
options = Options()
# 크롬드라이버 작동 
options.add_argument('--start-maximized') #전체화면
# options.add_argument('--headless') #headless(화면안켜고)
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#저장위치
blog_urls = dict()
blog_urls['dates'] = []
blog_urls['title'] = []
blog_urls['urls'] = []

#검색위치 
driver.get(base_url)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [ ]:
#_view_review_body_html > div > more-contents > div > ul

In [179]:
options = Options()
# 크롬드라이버 작동 
options.add_argument('--start-maximized') #전체화면
# options.add_argument('--headless') #headless(화면안켜고)
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#저장위치
blog_urls = dict()
blog_urls['dates'] = []
blog_urls['title'] = []
blog_urls['urls'] = []

#검색위치 
driver.get(base_url)

#최대 아래로 스크롤
count =1
string = '1'
some_tag = '1'
while type(some_tag) == type(string): #sele
    try : 
        some_tag = driver.find_element_by_id('sp_blog_1000')
    except:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        count += 1
print(count)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


34


In [250]:
options = Options()
# 크롬드라이버 작동 
options.add_argument('--start-maximized') #전체화면
options.add_argument('--headless') #headless(화면안켜고)
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#저장위치
blog_urls = dict()
blog_urls['dates'] = []
blog_urls['title'] = []
blog_urls['urls'] = []

#검색위치 
driver.get(base_url)

#최대 아래로 스크롤
count =1
string = '1'
some_tag = '1'
for i in range(0,40):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.1)
    count += 1
source = driver.page_source
# print(count)
driver.close()

#각 정보 불러오기
soup = BeautifulSoup(source, 'html.parser')

#dates
dates_list = soup.select('div > div.total_sub > span > span > span.etc_dsc_area > span')
dates = pd.Series(dates_list).apply(lambda x : x.text)
blog_urls['dates'] = blog_urls['dates'] + list(dates)

#titles
title_list = soup.select('div > a.api_txt_lines.total_tit')
titles = pd.Series(title_list).apply(lambda x : x.text)
blog_urls['title'] = blog_urls['title'] + list(titles)

#link
link_list = soup.select('div.total_dsc_wrap a.total_dsc')
links = pd.Series(link_list).apply(lambda x : x.get('href'))
blog_urls['urls'] = blog_urls['urls'] + list(links)


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [253]:
df = pd.DataFrame(blog_urls)

df_dict =  df.to_dict()

In [258]:
pd.DataFrame(df_dict)

,dates,title,urls
0,2020.12.17.,드라이브로 강릉 가볼만한곳 6,https://blog.naver.com/nailstan?Redirect=Log&l...
1,2020.12.25.,강릉 가볼만한곳 정동진부터 당일 5곳,https://blog.naver.com/tcacyc?Redirect=Log&log...
2,2020.12.29.,신선하게 즐겼던 강릉 횟집,https://dlfauddl.blog.me/222189422030
3,2020.12.31.,엄선해본 강원도 강릉 맛집 LIST,https://blog.naver.com/opop525?Redirect=Log&lo...
4,2020.12.29.,다채로웠던 강릉 맛집 추천 리스트,https://blog.naver.com/hangilchi?Redirect=Log&...
5,2020.12.26.,광고가 아닌 강릉 횟집 경포대 환호횟집,https://blog.naver.com/river0703?Redirect=Log&...
6,2020.12.26.,단번에 빠졌던 강릉횟집,https://blog.naver.com/llzzinll?Redirect=Log&l...
7,2020.12.26.,행복했던 강릉 횟집,https://blog.naver.com/amanda_som?Redirect=Log...
8,2020.11.28.,겨울에 강릉 가볼만한곳 모음,https://blog.naver.com/ultrabat?Redirect=Log&l...
9,2020.12.25.,강릉 회센터 회 맛집이구만!,https://blog.naver.com/amore_bh?Redirect=Log&l...


### 저장

In [232]:
# 본문가져오기
output_error = []
texts = []
for i, url in tqdm_notebook(enumerate(blog_urls['urls'])):
    count = i
    try:
        # 블로그 url안에 들어가기(아직 크롤링불가)
        get_blog_post_content_code = requests.get(url)
        get_blog_post_content_text = get_blog_post_content_code.text
        get_blog_post_content_soup = BeautifulSoup(
            get_blog_post_content_text, 'lxml')

        # 크롤링가능한 url에 접속
        real_blog_post_url = "http://blog.naver.com" + get_blog_post_content_soup.select('#mainFrame')[0].get('src')
        get_real_blog_post_content_code = requests.get(real_blog_post_url)
        get_real_blog_post_content_text = get_real_blog_post_content_code.text
        get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')

        # 본문  (postviewarea 패턴과 se-main-container 2가지 유형이 있어 분리함)
        try:
            blog_post_content = get_real_blog_post_content_soup.select('div.se-main-container')
            if len(blog_post_content) == 0:
                blog_post_content = get_real_blog_post_content_soup.select('div#postViewArea')
                if len(blog_post_content) == 0:
                    blog_post_content = get_real_blog_post_content_soup.select('div.se_component_wrap.sect_dsc.__se_component_area')

            string = ""
            for sentence in blog_post_content[0].stripped_strings:
                try :
                    string += " "+sentence.replace('\xa0', " ")
                except :
                    pass
                # 비언어 텍스트제거
            string = del_outword(string)
            # 공백에러대처
            blank_check = string.replace(" ", "")
            if len(blank_check) == 0:
                texts.append(["X"])
            else:
                texts.append([string])
            count += 1
        except Exception as ex:
            print('url_load_error')
            texts.append(["X"])
            output_error.append(count)
            count += 1

    except Exception as ex:
        print('가져오기에러 {num}번째'.format(num=count), ex)
        output_error.append(count)
        count += 1
        pass

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


가져오기에러 73번째 list index out of range
가져오기에러 93번째 list index out of range
가져오기에러 98번째 list index out of range
가져오기에러 99번째 list index out of range
가져오기에러 221번째 list index out of range
가져오기에러 235번째 list index out of range
가져오기에러 245번째 list index out of range
가져오기에러 259번째 list index out of range
가져오기에러 284번째 list index out of range
가져오기에러 314번째 list index out of range
가져오기에러 325번째 list index out of range
가져오기에러 336번째 list index out of range
가져오기에러 337번째 list index out of range
가져오기에러 358번째 list index out of range
가져오기에러 359번째 list index out of range
가져오기에러 419번째 list index out of range
가져오기에러 471번째 list index out of range
가져오기에러 480번째 list index out of range
가져오기에러 493번째 list index out of range
가져오기에러 518번째 list index out of range
가져오기에러 602번째 list index out of range
가져오기에러 628번째 list index out of range



In [233]:
len(blog_urls['dates'])

630

In [235]:
len(output_error)

22

In [236]:
len(texts)

608

In [158]:
#후처리 및 dataframe 생성
df = pd.DataFrame(blog_urls)
df = df.drop(output_error)
df['text'] = texts
lenght = df.shape[0]

#저장
if save == True:
    today = datetime.datetime.today().date()
    str(today)
    os.makedirs(f'./new_output/{keyword}',exist_ok=True)
    df.to_csv(f'./new_output/{keyword}/{date_from}~{date_to}_{keyword}_{lenght}_{today}.csv',index=False)